# MNIST - Handwriting Recognition

In [39]:
from com.yahoo.ml.caffe.DisplayUtils import *
from com.yahoo.ml.caffe.CaffeOnSpark import *
from com.yahoo.ml.caffe.Config import *
from com.yahoo.ml.caffe.DataSource import *

## Training Data

In [40]:
df = sqlCtx.read.parquet('/Users/afeng/dev/ml/CaffeOnSpark/data/mnist_test_dataframe')

In [41]:
show_df(df,5)

Index,Label,Image
00000000,7,
00000001,2,
00000002,1,
00000003,0,
00000004,4,


# Training

In [43]:
cos=CaffeOnSpark(sc,sqlContext)

In [44]:
args={}
args['conf']='/Users/afeng/dev/ml/CaffeOnSpark/data/lenet_dataframe_solver.prototxt'
args['model']='file:///tmp/lenet.model'
args['devices']='1'
args['clusterSize']='1'
cfg=Config(sc,args)

In [45]:
dl_train_source = DataSource(sc).getSource(cfg,True)

In [46]:
cos.train(dl_train_source)

# Test

In [47]:
dl_test_source = DataSource(sc).getSource(cfg,False)

In [48]:
test_result=cos.test(dl_test_source)

In [49]:
test_result

{u'accuracy': [0.9660000026226043], u'loss': [0.09910469958558679]}

# Feature Extraction

In [50]:
args['features']='accuracy,ip1,ip2'
args['label']='label'
cfg=Config(sc,args)

In [52]:
dl_feature_source = DataSource(sc).getSource(cfg,False)

In [54]:
f=cos.features(dl_feature_source)

In [55]:
f.show(5)

+--------+--------+--------------------+--------------------+-----+
|SampleID|accuracy|                 ip1|                 ip2|label|
+--------+--------+--------------------+--------------------+-----+
|00000000|   [1.0]|[-0.0, 3.109636, ...|[-0.6478175, -1.4...|[7.0]|
|00000001|   [1.0]|[1.3683326, -0.0,...|[2.0906663, 1.048...|[2.0]|
|00000002|   [1.0]|[1.5641443, -0.0,...|[-0.773368, 10.61...|[1.0]|
|00000003|   [1.0]|[-0.0, 1.9505613,...|[16.46351, -6.917...|[0.0]|
|00000004|   [1.0]|[0.5979191, 0.075...|[-0.48371825, -2....|[4.0]|
+--------+--------+--------------------+--------------------+-----+
only showing top 5 rows



In [56]:
def maxScoreAndIndex(array_of_scores): 
    return max(enumerate(array_of_scores), key=lambda x: x[1])
g = sqlContext.createDataFrame(f.map(lambda row: (
            row.SampleID,
            row.accuracy[0],
            row.ip2,
            maxScoreAndIndex(row.ip2)[1],
            maxScoreAndIndex(row.ip2)[0],
            int(row.label[0]))), 
            ["SampleID", "Accuracy", "Scores", "MaxScore", "Prediction", "Label"])
g.toPandas()[:5]

,SampleID,Accuracy,Scores,MaxScore,Prediction,Label
0,00000000,1.0,"[-0.647817492485, -1.41825199127, 4.0342922210...",17.381210,7,7
1,00000001,1.0,"[2.0906662941, 1.04856681824, 13.0640411377, 0...",13.064041,2,2
2,00000002,1.0,"[-0.773368000984, 10.6171503067, -1.5891783237...",10.617150,1,1
3,00000003,1.0,"[16.4635105133, -6.91782665253, 2.08401107788,...",16.463511,0,0
4,00000004,1.0,"[-0.483718246222, -2.35488677025, -1.850352764...",15.449157,4,4


###  Logistic Regression using MLlib

In [57]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

In [58]:
data = f.map(lambda row: LabeledPoint(row.label[0], Vectors.dense(row.ip1)))

In [59]:
lr = LogisticRegressionWithLBFGS.train(data, numClasses=10, iterations=10)

In [60]:
predictions = lr.predict(data.map(lambda pt : pt.features))

In [61]:
predictions.take(5)

[7, 2, 1, 0, 4]